In [1]:
import boto3
import sagemaker
import os

In [2]:
# session and role
sagemaker_session = sagemaker.Session()
role = sagemaker.get_execution_role()

# location of stored sata
data_dir = 'modelling'
# S3 location
bucket = sagemaker_session.default_bucket()
# set prefix, a descriptive name for a directory  
prefix = 'sagemaker/audio_emotion_detection'

# upload all data to S3
test_location = sagemaker_session.upload_data(os.path.join(data_dir, 'test.csv'), key_prefix=prefix)
train_location = sagemaker_session.upload_data(os.path.join(data_dir, 'train.csv'), key_prefix=prefix)
print('Test location: {}'.format(test_location))
print('Train location: {}'.format(train_location))

Test location: s3://sagemaker-eu-central-1-668160054588/sagemaker/audio_emotion_detection/test.csv
Train location: s3://sagemaker-eu-central-1-668160054588/sagemaker/audio_emotion_detection/train.csv


## Train estimator

In [3]:
s3_input_train = sagemaker.s3_input(s3_data=train_location, content_type='text/csv')
s3_input_test = sagemaker.s3_input(s3_data=test_location, content_type='text/csv')

output_path = 's3://{}/{}'.format(bucket, prefix)

## Load saved model

Steps to deploy a keras model on Sagemaker: https://aws.amazon.com/blogs/machine-learning/deploy-trained-keras-or-tensorflow-models-using-amazon-sagemaker/

In [ ]:
!ls export/Servo/tf_1

In [ ]:
#Tar the entire directory and upload to Amazon S3
import tarfile
with tarfile.open('model.tar.gz', mode='w:gz') as archive:
    archive.add('export', recursive=True)

inputs = sagemaker_session.upload_data(path='model.tar.gz', key_prefix=prefix)

## Deploy the trained model

In [ ]:
from sagemaker.tensorflow.model import TensorFlowModel
model = TensorFlowModel(model_data = 's3://' + bucket + '/' + key_prefix + '/' + 'model.tar.gz',
                        role = role,
                        framework_version = '2.0.0',
                        entry_point = 'predict.py',
                        source_dir='source')

In [ ]:
%%time
predictor = model.deploy(initial_instance_count=1,
                         instance_type='ml.m4.xlarge')